In [1]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ
##########################################
#    Aggregates SNR (.dat), latency (.dat), and ROI photos (png)
#            scraped from PhotoZ
#    Given ROI files in format slice#_rois.dat 

In [11]:
# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
data_dir = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/"
csv_prefix = 'scnn1a_all_slices.csv'
csv_columns = ['Date',
             'Slice/Loc/Rec',
             'Stim Location',
             'L4 barrel 1 Latency',
             'L4 barrel 2 Latency',
             'L4 barrel 3 Latency',
             'L4 barrel 1 SNR',
             'L4 barrel 2 SNR',
             'L4 barrel 3 SNR',
             'ROI jpg',
              'Age',
              'Infusion',
              'Sex',
              'Stim (uA)',
              'Delta latency 1 (barrel 1 - barrel 2)',
              'Delta latency 2 (barrel 3 - barrel 2)']

enable_photoZ_interaction = True
initialize_photoZ = False
snr_cutoff_signal = 5.0

# Load reference data for all slices
reference_file = "C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/scnn1a_catalog.csv"
reference_df = pd.read_csv(reference_file)
ref_cols = ['Age', 
          'Sex', 
          "Infusion", 
          'Stim (uA)']

# load the all-data CSV dir
df = pd.read_csv(data_dir + csv_prefix)
df = pd.DataFrame(df)
df

,Date,Slice/Loc/Rec,Stim Location,L4 barrel 1 Latency,L4 barrel 2 Latency,L4 barrel 3 Latency,L4 barrel 1 SNR,L4 barrel 2 SNR,L4 barrel 3 SNR,ROI jpg,Age,Infusion,Sex,Stim (uA),Delta latency 1 (barrel 1 - barrel 2),Delta latency 2 (barrel 3 - barrel 2)
0,2/28/2023,02_01_01,L5,50.3009,50.8406,49.3457,2.94039,10.63090,6.48439,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,NaN,-1.4949
1,2/28/2023,02_01_03,L5,50.6451,50.8291,49.8348,4.43717,9.88073,5.89912,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,NaN,-0.9943
2,2/28/2023,02_02_02,L5,49.9882,51.3212,51.1805,2.04521,5.14209,2.92133,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,NaN,NaN
3,2/28/2023,03_01_01,L2/3,52.5719,51.2136,52.9013,9.40509,10.23720,5.78093,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,1.3583,1.6877
4,2/28/2023,03_02_01,L2/3,51.6168,50.1283,50.9250,25.20450,27.82770,26.27210,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58,NaN,M,100,1.4885,0.7967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,6/9/2023,02_05_05,L2/3,52.8847,52.7759,52.7034,2.94845,3.03057,1.13669,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,64,NaN,M,200,NaN,NaN
81,6/9/2023,02_05_07,L2/3,52.6646,52.2427,52.9754,2.51584,3.37059,1.51257,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,64,NaN,M,200,NaN,NaN
82,6/9/2023,02_05_08,L2/3,54.0270,53.9975,59.4641,3.84072,3.43322,2.53465,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,64,NaN,M,200,NaN,NaN
83,6/9/2023,02_05_09,L2/3,52.8899,52.2646,53.6709,3.11165,3.64487,1.58509,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,64,NaN,M,200,NaN,NaN


In [3]:
# set up PhotoZ (open it manually)
if enable_photoZ_interaction:
    aPhz = AutoPhotoZ(data_dir=data_dir)
    if initialize_photoZ:
        aPhz.prepare_photoZ()

In [4]:
for subdir, dirs, files in os.walk(data_dir):
    zda_files = []
    roi_files = {}  # map slice number to ROIs file
    
    if 'selected_zda' not in subdir:
        continue
    
    # locate important file names
    for f in files:
        if f.endswith(".zda"):
            zda_files.append(f)
        if 'slice' in f[-17:] and f.endswith('_rois.dat'):
            slice_no = int(f.split("_rois.dat")[0][-1])
            roi_files[slice_no] = f
    
    if len(roi_files) < 1 or len(zda_files) < 1:
        continue
    
    for zda_file in zda_files:
        rec_id = zda_file.split('.')[0]
            
        # determine date and slice, loc, rec
        
        aPhz = AutoPhotoZ(data_dir=subdir)
        
        # open the PhotoZ file
        if enable_photoZ_interaction:
            aPhz.select_PhotoZ()
            
            aPhz.open_zda_file(subdir + "/" + zda_file)
        
        date = subdir.split("selected_zda")[0][-9:-1]
        date = [int(x) for x in date.split("-")]
        date[2] += 2000  # full year format
        date = "/".join([str(d) for d in date])
        
        slice_no = int(rec_id.split("_")[0])
        roi_file = roi_files[slice_no]
        print("\n\nscraping data from ", zda_file, "using", roi_file, "on date:", date)

        filename_end = "_" + rec_id + '.dat'

        # open roi file in photoZ
        if enable_photoZ_interaction:
            aPhz.select_roi_tab()
            aPhz.open_roi_file(roi_file)

        # save latency values from PhotoZ
        latency_filename = subdir + "/" + "latency_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_latency_trace_value()
            aPhz.save_trace_values(latency_filename)

        # save SNR values from PhotoZ
        snr_filename = subdir + "/" + "SNR_" + filename_end
        if enable_photoZ_interaction:
            aPhz.select_SNR_trace_value()
            aPhz.save_trace_values(snr_filename)


        # take JPG photo of ROIs so we know which measurements go where.
        rois_frame_filename = subdir + "/" + "ROIs_" + rec_id + ".jpg"
        if enable_photoZ_interaction:
            aPhz.save_map_jpeg(rois_frame_filename)

        intermed_files = [latency_filename, snr_filename, rois_frame_filename]
        column_groups = [
            ['L4 barrel 1 Latency', 'L4 barrel 2 Latency', 'L4 barrel 3 Latency'],
            ['L4 barrel 1 SNR', 'L4 barrel 2 SNR', 'L4 barrel 3 SNR'],
            ['ROI jpg']
        ]
        
        # look up stim location, age, sex, infusion from scnn1a_all.csv
        ref_row = reference_df[reference_df["Date"] == date][ref_cols]        
        new_row = {"Date" : date, 
                   "Slice/Loc/Rec": rec_id}
        for col in ref_cols:
            new_row[col] = ref_row.iloc[0][col]
        
        # open these various files and store them to the pandas dataframe
        # The first three ROIs of each should correspond to the 3 desired barrel ROIs
        # each barrel/value pair is a column in the target CSV
        for j in range(len(intermed_files)):
            if intermed_files[j].endswith(".dat"):
                intermed_df = pd.read_csv(intermed_files[j],
                                          sep='\t',
                                          header=None,
                                          names=['Index',  'Values'])
                for i in range(len(column_groups[j])):
                    lat_val = intermed_df['Values'][i]
                    new_row[column_groups[j][i]] = lat_val
            else:  # .jpg, store link as csv value
                new_row[column_groups[j][0]] = intermed_files[j]
        print(new_row)
        df = df.append(new_row, ignore_index=True)




scraping data from  02_01_01.zda using slice2_rois.dat on date: 2/8/2023
{'Date': '2/8/2023', 'Slice/Loc/Rec': '02_01_01', 'Age': 79, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200,150', 'L4 barrel 1 Latency': 50.2296, 'L4 barrel 2 Latency': 50.2176, 'L4 barrel 3 Latency': 53.0153, 'L4 barrel 1 SNR': 5.06973, 'L4 barrel 2 SNR': 7.08936, 'L4 barrel 3 SNR': 3.23049, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-08-23\\selected_zda/ROIs_02_01_01.jpg'}


scraping data from  02_01_02.zda using slice2_rois.dat on date: 2/8/2023
{'Date': '2/8/2023', 'Slice/Loc/Rec': '02_01_02', 'Age': 79, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200,150', 'L4 barrel 1 Latency': 49.047, 'L4 barrel 2 Latency': 49.4821, 'L4 barrel 3 Latency': 48.2181, 'L4 barrel 1 SNR': 3.3196, 'L4 barrel 2 SNR': 6.68988, 'L4 barrel 3 SNR': 1.7333, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-08-23\\selected_zda/ROIs_02_01_02.jpg'}


scraping data from  02_01_04.zda using slic

{'Date': '2/14/2023', 'Slice/Loc/Rec': '03_01_10', 'Age': 67, 'Sex': 'F', 'Infusion': 'NBQX', 'Stim (uA)': '120', 'L4 barrel 1 Latency': 50.2182, 'L4 barrel 2 Latency': 50.2766, 'L4 barrel 3 Latency': 51.9473, 'L4 barrel 1 SNR': 8.014, 'L4 barrel 2 SNR': 12.3195, 'L4 barrel 3 SNR': 3.77819, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-14-23\\selected_zda/ROIs_03_01_10.jpg'}


scraping data from  03_01_12.zda using slice3_rois.dat on date: 2/14/2023
{'Date': '2/14/2023', 'Slice/Loc/Rec': '03_01_12', 'Age': 67, 'Sex': 'F', 'Infusion': 'NBQX', 'Stim (uA)': '120', 'L4 barrel 1 Latency': 50.3696, 'L4 barrel 2 Latency': 50.3343, 'L4 barrel 3 Latency': 51.5514, 'L4 barrel 1 SNR': 10.4539, 'L4 barrel 2 SNR': 15.4922, 'L4 barrel 3 SNR': 4.03319, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/02-14-23\\selected_zda/ROIs_03_01_12.jpg'}


scraping data from  04_01_06.zda using slice4_rois.dat on date: 2/28/2023
{'Date': '2/28/2023', 'Slice/Loc/Rec': '04



scraping data from  01_02_06.zda using slice1_rois.dat on date: 6/9/2023
{'Date': '6/9/2023', 'Slice/Loc/Rec': '01_02_06', 'Age': 64, 'Sex': 'M', 'Infusion': nan, 'Stim (uA)': '200', 'L4 barrel 1 Latency': 53.0881, 'L4 barrel 2 Latency': 51.8998, 'L4 barrel 3 Latency': 53.9328, 'L4 barrel 1 SNR': 1.8017, 'L4 barrel 2 SNR': 3.15537, 'L4 barrel 3 SNR': 1.81538, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/06-09-23\\selected_zda/ROIs_01_02_06.jpg'}


scraping data from  01_03_04.zda using slice1_rois.dat on date: 6/9/2023
{'Date': '6/9/2023', 'Slice/Loc/Rec': '01_03_04', 'Age': 64, 'Sex': 'M', 'Infusion': nan, 'Stim (uA)': '200', 'L4 barrel 1 Latency': 54.658, 'L4 barrel 2 Latency': 54.0737, 'L4 barrel 3 Latency': 54.2511, 'L4 barrel 1 SNR': 1.56173, 'L4 barrel 2 SNR': 4.97484, 'L4 barrel 3 SNR': 3.71096, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/06-09-23\\selected_zda/ROIs_01_03_04.jpg'}


scraping data from  01_03_06.zda using slice1_rois



scraping data from  01_05_05.zda using slice1_rois.dat on date: 3/7/2023
{'Date': '3/7/2023', 'Slice/Loc/Rec': '01_05_05', 'Age': 168, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200', 'L4 barrel 1 Latency': 50.7989, 'L4 barrel 2 Latency': 48.2177, 'L4 barrel 3 Latency': 50.9546, 'L4 barrel 1 SNR': 9.14695, 'L4 barrel 2 SNR': 2.7924, 'L4 barrel 3 SNR': 3.92709, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/mm_hidden\\03-07-23\\selected_zda/ROIs_01_05_05.jpg'}


scraping data from  02_01_02.zda using slice2_rois.dat on date: 3/7/2023
{'Date': '3/7/2023', 'Slice/Loc/Rec': '02_01_02', 'Age': 168, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200', 'L4 barrel 1 Latency': 50.8362, 'L4 barrel 2 Latency': 49.9343, 'L4 barrel 3 Latency': 50.6991, 'L4 barrel 1 SNR': 5.16855, 'L4 barrel 2 SNR': 13.6424, 'L4 barrel 3 SNR': 5.80288, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/mm_hidden\\03-07-23\\selected_zda/ROIs_02_01_02.jpg'}


scraping data from  02_01



scraping data from  03_01_08.zda using slice3_rois.dat on date: 3/15/2023
{'Date': '3/15/2023', 'Slice/Loc/Rec': '03_01_08', 'Age': 176, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200', 'L4 barrel 1 Latency': 52.7915, 'L4 barrel 2 Latency': 51.0669, 'L4 barrel 3 Latency': 52.7473, 'L4 barrel 1 SNR': 8.87232, 'L4 barrel 2 SNR': 15.4697, 'L4 barrel 3 SNR': 13.7794, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/mm_hidden\\03-15-23\\selected_zda/ROIs_03_01_08.jpg'}


scraping data from  03_02_02.zda using slice3_rois.dat on date: 3/15/2023
{'Date': '3/15/2023', 'Slice/Loc/Rec': '03_02_02', 'Age': 176, 'Sex': 'F', 'Infusion': nan, 'Stim (uA)': '200', 'L4 barrel 1 Latency': 52.3772, 'L4 barrel 2 Latency': 50.6015, 'L4 barrel 3 Latency': 50.3944, 'L4 barrel 1 SNR': 5.93411, 'L4 barrel 2 SNR': 18.2544, 'L4 barrel 3 SNR': 21.4613, 'ROI jpg': 'C:/Users/jjudge3/Desktop/Data/mm_full_pipeline_targets/mm_hidden\\03-15-23\\selected_zda/ROIs_03_02_02.jpg'}


scraping data from  

In [12]:
# compute statistics on barrel pairs:
#    1) whether delta latency should be considered based on SNR
#    2) Delta latency between L4 barrels (b1 - b2, b3 - b2)
df['Delta latency 1 (barrel 1 - barrel 2)'] = df['L4 barrel 1 Latency'] - df['L4 barrel 2 Latency']
df['Delta latency 2 (barrel 3 - barrel 2)'] = df['L4 barrel 3 Latency'] - df['L4 barrel 2 Latency']

# if either barrel 1 or barrel 2 has low SNR, don't compute the delta latency between these two
df.loc[(df['L4 barrel 1 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 SNR'] < snr_cutoff_signal),
       'Delta latency 1 (barrel 1 - barrel 2)'] = None

# same for barrel 2 or barrel 3
df.loc[(df['L4 barrel 3 SNR'] < snr_cutoff_signal) | (df['L4 barrel 2 SNR'] < snr_cutoff_signal),
       'Delta latency 2 (barrel 3 - barrel 2)'] = None

for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    print("Barrel pairs in", stim_loc)
    # count number of barrel pairs with no signal crossing
    n_barrel_no_cross = len(df2[(df2['L4 barrel 1 SNR'] < snr_cutoff_signal) & 
                               (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 1 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 3 SNR'] > snr_cutoff_signal)])
    n_barrel_no_cross += len(df2[(df2['L4 barrel 3 SNR'] < snr_cutoff_signal) & 
                                (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    print("\t", n_barrel_no_cross, "no crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with signal crossing
    n_barrel_cross = len(df2[(df2['L4 barrel 1 SNR'] > snr_cutoff_signal) & 
                            (df2['L4 barrel 2 SNR'] > snr_cutoff_signal)])
    n_barrel_cross += len(df2[(df2['L4 barrel 2 SNR'] > snr_cutoff_signal) & 
                             (df2['L4 barrel 3 SNR'] > snr_cutoff_signal)])
    print("\t", n_barrel_cross, "crossing out of", len(df2) * 2, "in layer", stim_loc)

    # count number of barrel pairs with no signal
    n_barrel_none = len(df2[(df2['L4 barrel 1 SNR'] < snr_cutoff_signal) & 
                            (df2['L4 barrel 2 SNR'] < snr_cutoff_signal)])
    n_barrel_none += len(df2[(df2['L4 barrel 2 SNR'] < snr_cutoff_signal) & 
                             (df2['L4 barrel 3 SNR'] < snr_cutoff_signal)])
    print("\t", n_barrel_none, "no signal out of", len(df2) * 2, "in layer", stim_loc)

Barrel pairs in L2/3
	 3 no crossing out of 50 in layer L2/3
	 17 crossing out of 50 in layer L2/3
	 30 no signal out of 50 in layer L2/3
Barrel pairs in L4
	 38 no crossing out of 88 in layer L4
	 20 crossing out of 88 in layer L4
	 30 no signal out of 88 in layer L4
Barrel pairs in L5
	 8 no crossing out of 22 in layer L5
	 13 crossing out of 22 in layer L5
	 1 no signal out of 22 in layer L5


In [46]:
df['abs(delta 1)'] = np.abs(df['Delta latency 1 (barrel 1 - barrel 2)'])
df['abs(delta 2)'] = np.abs(df['Delta latency 2 (barrel 3 - barrel 2)'])



for stim_loc in ['L2/3', 'L4', 'L5']:
    df2 = df[df['Stim Location'].str.contains(stim_loc)]
    dfd1 = df2['abs(delta 1)'].dropna()
    dfd2 = df2['abs(delta 2)'].dropna()

    print("Latency between barrel pairs in", stim_loc)
    dfds = np.concatenate([dfd1, dfd2])
    print("\t", 
          str(np.mean(dfds))[:5], 
          "+/-", 
          str(np.std(dfds))[:5], 
          "(n =", 
          str(len(dfds)) + ")")
    print("\t", dfds)

Latency between barrel pairs in L2/3
	 1.098 +/- 0.542 (n = 17)
	 [1.3583 1.4885 1.8089 0.5078 0.5782 0.8922 0.7235 1.1776 0.4841 1.8474
 1.6877 0.7967 0.7226 1.9021 0.7541 0.2124 1.7317]
Latency between barrel pairs in L4
	 0.846 +/- 1.106 (n = 20)
	 [0.1687 1.0151 1.3033 2.096  0.826  0.012  3.7841 0.5839 0.456  0.8266
 0.477  0.2671 0.0656 0.0797 0.0584 0.0353 0.2154 3.7901 0.03   0.8443]
Latency between barrel pairs in L5
	 0.726 +/- 0.446 (n = 13)
	 [0.8933 0.1239 1.4173 0.5098 0.0111 1.4949 0.9943 0.784  0.5802 1.0574
 0.1154 0.724  0.7415]


In [7]:
# write the pandas dataframe back to csv
csv_filename = 'scnn1a_all_slices-edited.csv'
df.to_csv(data_dir + csv_filename, index=False)

In [10]:
df

,Date,Slice/Loc/Rec,Stim Location,L4 barrel 1 Latency,L4 barrel 2 Latency,L4 barrel 3 Latency,L4 barrel 1 SNR,L4 barrel 2 SNR,L4 barrel 3 SNR,ROI jpg,Age,Infusion,Sex,Stim (uA),Delta latency 1 (barrel 1 - barrel 2),Delta latency 2 (barrel 3 - barrel 2)
0,2/28/2023,02_01_01,L5,50.3009,50.8406,49.3457,2.94039,10.63090,6.48439,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58.0,NaN,M,100,NaN,-1.4949
1,2/28/2023,02_01_03,L5,50.6451,50.8291,49.8348,4.43717,9.88073,5.89912,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58.0,NaN,M,100,NaN,-0.9943
2,2/28/2023,02_02_02,L5,49.9882,51.3212,51.1805,2.04521,5.14209,2.92133,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58.0,NaN,M,100,NaN,NaN
3,2/28/2023,03_01_01,L2/3,52.5719,51.2136,52.9013,9.40509,10.23720,5.78093,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58.0,NaN,M,100,1.3583,1.6877
4,2/28/2023,03_02_01,L2/3,51.6168,50.1283,50.9250,25.20450,27.82770,26.27210,C:/Users/jjudge3/Desktop/Data/mm_full_pipeline...,58.0,NaN,M,100,1.4885,0.7967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
